In [0]:
 spark = SparkSession \
    .builder 
    .appName("Spark Pracitcal")
    .config()
    .getOrCreate()


  File <command-4126949813681305>:3
    .appName("Spark Pracitcal")
    ^
IndentationError: unexpected indent


In [0]:
df = spark.read.csv("dbfs:/FileStore/flight_info.csv", header=True, inferSchema=True)

In [0]:
df.show()
# or 
# df.take(1)

In [0]:
df.columns

In [0]:
df.select("UniqueCarrier").show()

In [0]:
df.count()

In [0]:
df.select("UniqueCarrier").distinct()

In [0]:
df.select("UniqueCarrier").show()

In [0]:
df.select("UniqueCarrier").distinct().count()

In [0]:
df.select(["CRSDepTime", "DepTime"]).head(5)

In [0]:
my_filter = df["DepTime"] > df["CRSDepTime"]
my_filter


In [0]:
df.filter(my_filter)

In [0]:
df.count()

In [0]:
df.filter(my_filter).count()

In [0]:
my_filter = (df["DepTime"] - df["CRSDepTime"]) > 10
df.filter(my_filter).count()


In [0]:
df.select("UniqueCarrier").filter(my_filter).groupBy("UniqueCarrier").count().show()


In [0]:

df.select("UniqueCarrier").filter(my_filter).groupBy("UniqueCarrier").count().sort("count", ascending=False).show()



In [0]:
delay_counts_df = df.select("UniqueCarrier").filter(my_filter).groupBy("UniqueCarrier").count().sort("count", ascending=False)


In [0]:
airlines_df = spark.read.csv("dbfs:/FileStore/airlines.csv", header=False, inferSchema=True)

In [0]:
airlines_df.head(3)

In [0]:
delay_counts_df.join(airlines_df).show()

In [0]:
delay_counts_df.join(airlines_df).count()

In [0]:
delay_counts_df.count()

In [0]:
airlines_df.count()

In [0]:
6161 * 12 

In [0]:
delay_counts_df.join?

In [0]:
delay_counts_df.join(airlines_df, delay_counts_df["UniqueCarrier"] == airlines_df["_c3"]).show()

In [0]:
delay_counts_df.join(airlines_df.select(["_c3", "_c5"]), delay_counts_df["UniqueCarrier"] == airlines_df["_c3"]).show()

In [0]:
df.select("UniqueCarrier", "DayOfWeek").show()

In [0]:
df.select("UniqueCarrier", "DayOfWeek").filter(my_filter).groupBy("UniqueCarrier", "DayOfWeek").count().show()

In [0]:
df.select("UniqueCarrier", "DayOfWeek").filter(my_filter).groupBy("UniqueCarrier", "DayOfWeek").count().sort(["DayOfWeek", "count"], ascending=[1,0]).show()

In [0]:
temp = df.select("UniqueCarrier", "DayOfWeek").filter(my_filter).groupBy("UniqueCarrier", "DayOfWeek").count().sort(["DayOfWeek", "count"], ascending=[1,0])

In [0]:
temp.select( "DayOfWeek", "count", "UniqueCarrier").filter(temp.DayOfWeek == 7).show()

In [0]:
total_nb_flights_df = df.select("UniqueCarrier").groupBy("UniqueCarrier").count()


In [0]:
total_nb_flights_df.show()

In [0]:

delay_counts_df.show()


In [0]:
total_nb_flights_df.join?

In [0]:
join_delays_with_totals = total_nb_flights_df.join(delay_counts_df, on="UniqueCarrier", how="inner")

In [0]:
join_delays_with_totals.show()

In [0]:
join_delays_with_totals_renamed = join_delays_with_totals.withColumnRenamed("UniqueCarrier", "Carrier") \
            .withColumnRenamed("count", "flightCount").withColumnRenamed("count", "delayCount")

join_delays_with_totals_renamed.show()

In [0]:
total_nb_flights_df = df.select("UniqueCarrier").groupBy("UniqueCarrier").count().withColumnRenamed("count", "flightCount")
total_nb_flights_df.show()


In [0]:
join_delays_with_totals = total_nb_flights_df.join(delay_counts_df, on="UniqueCarrier", how="inner")
join_delays_with_totals.show()

In [0]:
join_delays_with_totals_renamed = join_delays_with_totals.withColumnRenamed("count", "delayCount")

join_delays_with_totals_renamed.show()

In [0]:
normalized_delay_count = join_delays_with_totals_renamed.withColumn("normalized_count", 
                                 join_delays_with_totals_renamed.delayCount / join_delays_with_totals_renamed.flightCount)

In [0]:
normalized_delay_count.sort("normalized_count", ascending=False).show()